In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import io
import PIL

In [ ]:
def gen_plot_batch(ts_batch):
    """Create a pyplot plot and save to buffer."""
    plt.figure()
    for batch_elmt in ts_batch:
        plt.plot(batch_elmt)
    plt.title("test")
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = PIL.Image.open(buf)
    ar = np.asarray(image)
    buf.close()
    plt.close()
    return ar

In [ ]:
def gen_plot_single(ts):
    """Create a pyplot plot and save to buffer."""
    plt.figure()
    plt.plot(ts)
    plt.title("test")
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = PIL.Image.open(buf)
    ar = np.asarray(image)
    buf.close()
    plt.close()
    return ar

In [ ]:
import tensorflow as tf
import numpy as np
import time
import os
import sys

for target_stock in target_stocks:

    with tf.variable_scope(target_stock):
        # tf Graph input
        x_tf = tf.placeholder("float", [batch_size, seq_length, dim_inputs])
        x_placeholders[target_stock] = x_tf

        y, a, b = build_invariant_network(x_tf, depth_list, non_linearity_list,
                                          seq_length, batch_size, dim_inputs, dim_outputs)

        list_of_as.append(a)
        list_of_bs.append(b)

        y = tf.reshape(y, [batch_size, -1])

        outputs = [tf.reshape(x, [-1]) for x in tf.split(0, batch_size, y)]

        features.append(y)

linear_combination = tf.pack(list_of_as)
residual = tf.add_n(features)

mean_residual = tf.reduce_mean(residual)
mean_residual_h = tf.scalar_summary("mean residual", mean_residual)

centered_residuals = residual - mean_residual
rho_1 = tf.reduce_sum(centered_residuals[:, 1:] * centered_residuals[:, :-1])
rho_1 = rho_1 / tf.reduce_sum(centered_residuals * centered_residuals)
rho_1_h = tf.scalar_summary("rho_1", rho_1)

cost = tf.nn.l2_loss(tf.add_n(outputs))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
cost_h = tf.scalar_summary("total_cost", cost)

summaries = tf.merge_all_summaries()

########################################################
#
#   Supplementary summaries for performance evaluation
#
########################################################

plot_placehoder = tf.placeholder("uint8", [None, None, None, 4])
plot_h = tf.image_summary("Cointegration eval", plot_placehoder, max_images=batch_size)


for i in xrange(batch_size):
    next_batch_array = np.asanyarray([next_batch[x][i] for x in stock_list])
    values[i], signals[i] = RunCointegration(next_batch_array, next_batch_array, lin_comb, residual_eval[i],
                                             bot_type, bot_args)

    next_batch_array = np.asanyarray([next_batch[x][i] for x in stock_list])
    plot_result = plot_cointegration(next_batch_array, stock_list,
                                     residual_eval[i], values[i], signals[i])
    plot_results.append(plot_result)

plot_summary_eval = sess.run(plot_h, feed_dict={plot_placehoder: np.asanyarray(plot_results)})
test_summary_writer.add_summary(plot_summary_eval, step)

test_summary_writer.add_summary(sharpe_ratio_summary_eval, step)

print("Iter " + str(step*batch_size) + ", Minibatch Loss= " +
      "{:.6f}".format(loss) + ", Minibatch Test Loss= " + "{:.6f}".format(loss_test))

step += 1
print("Optimization Finished!")